In [68]:
from elasticsearch import Elasticsearch, helpers
from elasticsearch_dsl import Search, Q

import csv


In [7]:
es = Elasticsearch(HOST="localhost", PORT="9200")

In [8]:
faq_csv = ".data/faq_with_splits_lemmatized.csv"

In [ ]:
es.indices.delete(index='questions', ignore=[404])

with open(faq_csv) as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='questions')

In [10]:
def generate_query(question):
    return {
        "match": {
            "long_question": {
                "query": question,
            }
        }
    }
    
def find_answer(query_param, generate_query, index="questions"):
    query = generate_query(query_param)
    res = es.search(index=index, query=query)
    
    return res['hits']['hits'][0]['_source']

In [24]:
question = input()

result =  find_answer(question, generate_query)

print(f"Short question:\n{result['short_question']}")
print()
print(f"Long question:\n{result['long_question']}")
print()
print(f"Answer:\n{result['answer']}")

miért fáj láb
Short question:
láb varratszedés után mennyi idő után mehet dolgoz ?

Long question:
Láb varratszedés után mennyi idő után mehetek dolgozni? Mennyire javul meg a láb varratszedés után? Mehetek másnap dolgozni vagy a mozgás azért nehezebben megy még?

Answer:
Nem lehet megmondani. Az orvosod tud erre válaszolni. Nem mindegy mekkora a seb, és milyen célból történt a műtét?


In [157]:
INDEX_NAME = "questions"
INDEX_FILE = "index.json"
DATA_FILE = faq_csv
BATCH_SIZE = 1000

In [158]:
import hu_core_ud_lg
import numpy as np

import pandas as pd

In [159]:
data_frame = pd.read_csv(DATA_FILE)

In [160]:
nlp = hu_core_ud_lg.load()

In [161]:
def embed_docs(docs):
    return [embed(doc) for doc in docs]

In [162]:
def embed(sentence):
    doc = nlp(sentence)
    helper = [doc[idx].vector for idx in range(len(doc))]
    query_vector = np.add.reduce(helper) / len(doc)
    
    return query_vector

In [163]:
header_k2i = {}
header_i2k = {}

def handle_header(header):
    for idx, key in enumerate(header):
        header_k2i[key] = idx
        header_i2k[idx] = key

In [170]:
def index_data(df):
    es.indices.delete(index=INDEX_NAME, ignore=[404])

    with open(INDEX_FILE) as index_file:
        source = index_file.read().strip()
        es.indices.create(index=INDEX_NAME, body=source)
    
    docs = []
    count = 0
    
    for index, row in df.iterrows():
        docs.append(row.to_dict())
        count += 1
            
        if count % BATCH_SIZE == 0:
            index_batch(docs)
            docs = []
            print(f"\rIndexed {count} documents.", end="")

    if docs:
        index_batch(docs)
        docs = []
        print(f"\rIndexed {count} documents.", end="")

    print()
    es.indices.refresh(index=INDEX_NAME)
    print("Done indexing.")

In [171]:
def index_batch(docs):
    short_questions = [doc["short_question"] for doc in docs]
    
    short_question_vectors = embed_docs(short_questions)

    requests = []
    
    for i, doc, in enumerate(docs):
        request = doc
        request["_op_type"] = "index"
        request["_index"] = INDEX_NAME
        request["short_question_vector"] = short_question_vectors[i]
        requests.append(request)
    helpers.bulk(es, requests)

In [172]:
index_data(data_frame)

/tmp/ipykernel_30071/839224222.py:6: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=source)


Indexed 168126 documents.
Done indexing.


In [ ]:
script_query = {
    "script_score": {
        "query": {"match_all": {}},
        "script": {
            "source": "cosineSimilarity(params.query_vector, doc['short_question_vector']) + 1.0",
            "params": {"query_vector": embed(question)}
        }
    }
}

response = es.search(
    index=INDEX_NAME,
    body={
        "size": 10,
        "query": script_query
    }
)

In [176]:
for hit in response["hits"]["hits"]:
    print("id: {}, score: {}".format(hit["_id"], hit["_score"]))
    print(hit["_source"]["short_question"])
    print(hit["_source"]["answer"])
    print()

id: uqRjTXwBNIJMXm8Ye_is, score: 1.8774538
miért fáj a kutya láb ?
Vidd már orvoshozte idióta!!

id: 1qJZTXwBNIJMXm8YnOaL, score: 1.8579246
miért fáj a bal láb menzeszkor ?
Ne aggódj.nekem is mindig fájtak a lábaim menseskor.Sajnos lesugárzik a fájdalom.

id: iqNaTXwBNIJMXm8YfBa2, score: 1.8389784
miért fáj fark ?
Nem az erős kézfogással kell a hatást elérni .Akkor nem fordúl elő ilyen .Egy erős pofontól az arcod is piros lesz,és még meg is fájdul .Na ez van a pö**öddel is .

id: _6NaTXwBNIJMXm8Y_DHX, score: 1.8027391
miért fáj még mindig a láb ?
Legalább valamilyen nadálytő krémmel, v. lóbalzsammal msszírozhattad és éjjelre pakolás. Így "szárazon" sokáig gyógyul.

id: RaRjTXwBNIJMXm8Yavcd, score: 1.8011953
fáj a kutya láb ?
ha valami eltörik, ha meggyógyult is, mindig érzékeny marad. Általában időjárás változásnál jelentkeznek fájdalmak. Beszélj az orvosoddal és ő ajánlani fog gyógyszert, amit akkor adhatsz a kutyádnak, amikor jelzi, hogy fáj a lába.

id: 4aNbTXwBNIJMXm8YjU8z, score: 